In [ ]:
#summarization deploy 
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'facebook/bart-large-cnn',
	'HF_TASK':'summarization'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.37.0',
	pytorch_version='2.1.0',
	py_version='py310',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

predictor.predict({
	"inputs": "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.",
})

In [ ]:
#summarization endpoint call 
import boto3
import json

# SageMaker Runtime 클라이언트 생성
runtime = boto3.client("sagemaker-runtime", region_name="ap-northeast-2")  # 서울 리전 예시

# 엔드포인트 이름
endpoint_name = "huggingface-pytorch-inference-2025-05-14-10-53-09-680"  # 여기에 본인의 엔드포인트 이름을 넣으세요

# 입력 데이터 (예시: HuggingFace-style JSON)
input_data = {
    "inputs": "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."
}

# JSON 직렬화
payload = json.dumps(input_data)

# Endpoint 호출
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",  # 대부분의 NLP 모델은 JSON 형식 사용
    Body=payload
)

# 결과 디코딩
result = response["Body"].read().decode("utf-8")

# 출력 결과 확인
print("Inference result:")
print(result)



In [ ]:
#translation model ko-> en deploy
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'facebook/nllb-200-distilled-600M',
	'HF_TASK':'translation'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.37.0',
	pytorch_version='2.1.0',
	py_version='py310',
	env=hub,
	role=role,
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge', # ec2 instance type
    src_lang = "ko",   # 소스 언어: 한국어(ISO 639-3 코드)
    tgt_lang = "en",
)

predictor.predict({'inputs': """트럼프 미국 대통령이 캐나다와 멕시코에 대한 관세를 약 한 달간 유예하기로 했지만, 관세 정책을 둘러싼 불확실성이 투자 심리를 짓누르면서 뉴욕 증시의 3대 주가지수가 동반 하락했습니다.

뉴욕 증권 거래소에서 나스닥 종합 지수는 전장보다 483.48포인트, 2.61% 주저앉은 18,069.26에 장을 마쳤습니다.

스탠더드 앤드 푸어스 500지수는 104.11포인트, 1.78% 급락한 5,738.52에, 다우존스30산업평균지수는 427.51포인트, 0.99% 떨어진 42,579.08로 거래를 마감했습니다.

이는 트럼프가 미국·멕시코·캐나다 무역 협정 적용 품목에 다음 달 2일까지 25% 관세를 면제하기로 했지만, 투자자들은 정책 방향이 너무 가변적이라는 데 불안을 느낀 것으로 풀이됩니다.

또 헤지펀드 설립자로, 트럼프의 정책에 제동을 걸어줄 것으로 기대됐던 스콧 베센트 재무장관이 캐나다 총리를 비하하는 등 트럼프와 유사한 어법을 구사하자 월가는 실망하는 분위기였습니다.
""",
                   'parameters': {'src_lang': 'ko', 'tgt_lang' : 'en'}})





In [ ]:
#trainslation call ko->en
import boto3
import json

# SageMaker Runtime 클라이언트 생성
runtime = boto3.client("sagemaker-runtime", region_name="ap-northeast-2")  # 리전은 자신의 것으로 수정

# 엔드포인트 이름
endpoint_name = "huggingface-pytorch-inference-2025-05-14-10-58-26-686"

# 입력 구성
input_payload = {
    "inputs": """트럼프 미국 대통령이 캐나다와 멕시코에 대한 관세를 약 한 달간 유예하기로 했지만, 관세 정책을 둘러싼 불확실성이 투자 심리를 짓누르면서 뉴욕 증시의 3대 주가지수가 동반 하락했습니다.

뉴욕 증권 거래소에서 나스닥 종합 지수는 전장보다 483.48포인트, 2.61% 주저앉은 18,069.26에 장을 마쳤습니다.

스탠더드 앤드 푸어스 500지수는 104.11포인트, 1.78% 급락한 5,738.52에, 다우존스30산업평균지수는 427.51포인트, 0.99% 떨어진 42,579.08로 거래를 마감했습니다.

이는 트럼프가 미국·멕시코·캐나다 무역 협정 적용 품목에 다음 달 2일까지 25% 관세를 면제하기로 했지만, 투자자들은 정책 방향이 너무 가변적이라는 데 불안을 느낀 것으로 풀이됩니다.

또 헤지펀드 설립자로, 트럼프의 정책에 제동을 걸어줄 것으로 기대됐던 스콧 베센트 재무장관이 캐나다 총리를 비하하는 등 트럼프와 유사한 어법을 구사하자 월가는 실망하는 분위기였습니다.
""",
    "parameters": {
        "src_lang": "ko",
        "tgt_lang": "en"
    }
}

# JSON 직렬화
payload = json.dumps(input_payload)

# Endpoint 호출
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

# 결과 디코딩
result = response['Body'].read().decode('utf-8')
print("Prediction result:")
print(result)


In [ ]:
#translation model en-> ko deploy
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'facebook/nllb-200-distilled-600M',
	'HF_TASK':'translation'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.37.0',
	pytorch_version='2.1.0',
	py_version='py310',
	env=hub,
	role=role,
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.2xlarge', # ec2 instance type
    src_lang = "en",   # 소스 언어: 한국어(ISO 639-3 코드)
    tgt_lang = "ko",
)

predictor.predict({'inputs': """the President of the United States, had postponed tariffs on Canada and Mexico for about a month, but the uncertainty surrounding the tariff policy had crushed the investment core, and the 3rd largest indicator of the New York Stock Exchange had collapsed along with it.The NASDAQ index on the New York Stock Exchange ended the chapter at 483.48 points above the previous one, 2.61% below the 18,069.26 point.The Standard & Poor's 500 index was 104.11 points, 1.78% below the 5,738.52 point, while Dow Jones's 30 business index closed the deal at 427.51 points, 0.99% below the 579.08 point.This led to Trump's decision to exempt from 42,25% on items that applied to US-Mexico-Canada trade until the next 2 days, investors felt uncertainty that the direction of the trade would be moving forward.
""",
                   'parameters': {'src_lang': 'en', 'tgt_lang' : 'ko'}})





In [ ]:
#trainslation call en->ko
import boto3
import json

# SageMaker Runtime 클라이언트 생성
runtime = boto3.client("sagemaker-runtime", region_name="ap-northeast-2")  # 리전은 자신의 것으로 수정

# 엔드포인트 이름
endpoint_name = "huggingface-pytorch-inference-2025-05-14-11-14-29-162"

# 입력 구성
input_payload = {
    "inputs": """Trump, the President of the United States, had postponed tariffs on Canada and Mexico for about a month, but the uncertainty surrounding the tariff policy had crushed the investment core, and the New York Stock Exchange's 3rd largest indicator had collapsed along with it.The NASDAQ index on the New York Stock Exchange ended the chapter at 483.48 points above the previous one, 2.61% below the 18,069.26 point.The Standard & Poor's 500 index was 104.11 points, 1.78% below the 5,738.52 point, while Dow Jones's 30 business index closed the deal at 427.51 points, 0.99% below the 42,579.08 point.This led to Trump's decision to exempt from tariffs of 25% on items that applied to US-Mexico-Canada trade until the next 2 days, but investors felt uncertainty that the direction of the
""",
    "parameters": {
        "src_lang": "eng_Latn",
        "tgt_lang": "kor_Hang"
    }
}

# JSON 직렬화
payload = json.dumps(input_payload)

# Endpoint 호출
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

# 결과 디코딩
result = response['Body'].read().decode('utf-8')
print("Prediction result:")
print(result)


In [ ]:
pip install openai

In [ ]:
#추천 
import openai
import torch
import torch.nn.functional as F

# ✅ OpenAI API 키 설정 (환경변수로 설정해도 됨)
openai.api_key = ""  # 안전하게 환경변수로 관리하는 것을 권장합니다.

# ✅ 임베딩 함수 정의
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

def get_batch_embeddings(texts, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=texts,
        model=model
    )
    return [item.embedding for item in response.data]

# ✅ 입력 데이터
texts = [
    "Illustration of the REaLTabFormer model...",
    "Predicting human mobility holds significant...",
    "As the economies of Southeast Asia continue..."
    # 총 10개 문장까지 확장 가능
]

liked_texts = [
    "I enjoy reading about advanced machine learning models and their applications.",
    "Human mobility prediction is fascinating, especially when using transformer-based models."
]

# ✅ liked 임베딩 평균 계산
liked_embeddings = get_batch_embeddings(liked_texts)
liked_tensor = torch.tensor(liked_embeddings)
mean_liked_embedding = torch.mean(liked_tensor, dim=0)

# ✅ texts 임베딩 계산
text_embeddings = get_batch_embeddings(texts)
text_tensors = torch.tensor(text_embeddings)

# ✅ 코사인 유사도 계산
similarities = F.cosine_similarity(text_tensors, mean_liked_embedding.unsqueeze(0), dim=1)

# ✅ 가장 유사한 문장 추출
most_similar_index = torch.argmax(similarities).item()
most_similar_sentence = texts[most_similar_index]

# ✅ 출력
print(f"\n📌 Most similar sentence:\n{most_similar_sentence}")
print(f"🔗 Similarity score: {similarities[most_similar_index].item():.4f}")

# ✅ 모든 유사도 출력
print("\n🔎 All similarity scores:")
for i, (text, score) in enumerate(zip(texts, similarities)):
    print(f"Sentence {i+1}: {score.item():.4f} - {text[:60]}...")
